# 🖼️ CIFAR-10 Classification: Scratch CNN vs ResNet18

This notebook compares two approaches for CIFAR-10 image classification:
- **Scratch CNN**: A custom CNN trained from scratch (~75% accuracy).
- **ResNet18 (pretrained)**: Transfer learning with ImageNet-pretrained weights (~90% accuracy).

We will:
1. Load and preprocess CIFAR-10 dataset.
2. Train a simple CNN from scratch.
3. Fine-tune a pretrained ResNet18.
4. Evaluate and visualize results.
5. Compare performance and interpretability.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

## 1. Load CIFAR-10 Dataset
- 60,000 color images (32x32)
- 10 classes: airplane, car, bird, cat, deer, dog, frog, horse, ship, truck

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=100, shuffle=False)

classes = trainset.classes
print("Classes:", classes)

## 2. Scratch CNN Model

In [ ]:
class ScratchCNN(nn.Module):
    def __init__(self):
        super(ScratchCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.fc1 = nn.Linear(128*4*4, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

scratch_model = ScratchCNN().to(device)
print(scratch_model)

## 3. ResNet18 (Pretrained on ImageNet)
We replace the final fully connected layer to output 10 CIFAR-10 classes.

In [ ]:
import torchvision.models as models

resnet18 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
resnet18.fc = nn.Linear(resnet18.fc.in_features, 10)
resnet18 = resnet18.to(device)
print(resnet18)

## 4. Training Function (shared by both models)

In [ ]:
def train_model(model, trainloader, testloader, epochs=10, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    train_losses, train_accs, test_accs = [], [], []
    
    for epoch in range(epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        train_losses.append(running_loss/len(trainloader))
        train_accs.append(100*correct/total)
        
        # Evaluation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
        test_accs.append(100*correct/total)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {train_losses[-1]:.4f}, Train Acc: {train_accs[-1]:.2f}%, Test Acc: {test_accs[-1]:.2f}%")
    return train_losses, train_accs, test_accs

## 5. Train Scratch CNN (20 epochs)

In [ ]:
scratch_losses, scratch_train_accs, scratch_test_accs = train_model(scratch_model, trainloader, testloader, epochs=20, lr=0.001)

## 6. Train ResNet18 (15 epochs)

In [ ]:
resnet_losses, resnet_train_accs, resnet_test_accs = train_model(resnet18, trainloader, testloader, epochs=15, lr=0.0001)

## 7. Compare Results
- Loss and accuracy curves
- Final test accuracies
- Confusion matrices
- Grad-CAM visualizations